#*Importing libraries*

In [ ]:
import numpy as np

In [ ]:
from scipy.sparse import identity, csr_matrix, kron, hstack, vstack

#*The main code of the Lifted Product code*

In [ ]:
class LiftedProductCode:

    # Initialize with cyclic group order n (size of the lifting group G)
    def __init__(self, group_order):
        self.n = group_order
        self.G = self._generate_group()

    # Generate the cyclic group matrices G = [I, P, P^2, ..., P^(n-1)] in sparse form
    def _generate_group(self):
        P = csr_matrix(np.roll(np.eye(self.n), shift=1, axis=1))
        return [P ** i for i in range(self.n)]

    #Lifting the matrices with the group
    def lifted_matrix(self, base_matrix):
        lifted_rows = []

        for row in base_matrix:
            block_row = []
            for val in row:
                if isinstance(val, int) and val != 0:
                    block = self.G[val % self.n]
                else:
                    block = csr_matrix((self.n, self.n))
                block_row.append(block)
            lifted_rows.append(hstack(block_row))

        return vstack(lifted_rows)

    # Compute the lifted product code for matrices A and B
    def compute_product_code(self, A, B):
        A_lift = self.lifted_matrix(A)
        B_lift = self.lifted_matrix(B)

        mA, nA = A_lift.shape
        mB, nB = B_lift.shape

        HX_left = kron(A_lift, identity(mB, format='csr'), format='csr')
        HX_right = kron(identity(mA, format='csr'), B_lift, format='csr')
        HX = hstack([HX_left, HX_right], format='csr')

        HZ_left = kron(identity(nA, format='csr'), B_lift.T, format='csr')
        HZ_right = kron(A_lift.T, identity(nB, format='csr'), format='csr')
        HZ = hstack([HZ_left, HZ_right], format='csr')

        return HX, HZ

#*Accessing the .txt file for H1 matrix and getting the matrix*

In [ ]:
nums_list = []

with open('/content/H1_Marix_100n_50k.txt', 'r') as file:
    for line in file:
        numbers = list(map(int, line.strip().split()))
        nums_list.append(numbers)

nums = np.array(nums_list, dtype=int)
nums = np.matrix(nums)

In [ ]:
rows = 100
cols = 50
H1 = np.zeros((rows, cols))

In [ ]:
num_rows, num_cols = nums.shape
for i in range(num_rows):
  for j in range(num_cols):
    if nums[i,j] != -1:
      H1[i, nums[i,j]] = 1
    else:
      continue

In [ ]:
np.shape(H1)

(100, 50)

#*Preparing the H2 matrix*

*H2 is the parity check matrix of the 3-bit repetition code*

In [ ]:
H2 = np.array([[1, 1, 0], [0, 1, 1]])

#*Getting the final parity check matrix of the Lifted Product code*

In [ ]:
lpc = LiftedProductCode(group_order=50)

HX, HZ = lpc.compute_product_code(H1, H2)

if HX is not None:
    print("H_X shape:", HX.shape)
    print("H_Z shape:", HZ.shape)

H_X shape: (500000, 1000000)
H_Z shape: (375000, 1000000)
